In [11]:
#import libraries
import pandas as pd
import random #use to generate data

from datetime import datetime, timedelta

In [12]:
# --- Constants and Configuration ---
CROP_VALUES = {
    '1': 3000, '2': 2500, '3': 2000,
    '4': 3500, '5': 1800, '6': 1500
}
JUNE_CROPS = [1,2,3,4] # 1 = 'Soja', 2 = 'Milho', 3 = 'Arroz', 4 = 'Feijão'
JANUARY_CROPS = [5,6] # 5 = 'Trigo', 6 = 'Aveia'
ALL_CROPS = JUNE_CROPS + JANUARY_CROPS
LOAN_TERMS = [180, 360] # Updated loan terms as per new requirements
MAX_LOANS_PER_YEAR = 5
MIN_LOAN_INTERVAL_DAYS = 30 # Minimum days between loans for the same client

# Hectar Constants
MIN_TOTAL_HECTAR_CLIENT = 50  # Minimum total hectares a client might own
MAX_TOTAL_HECTAR_CLIENT = 1000 # Maximum total hectares a client might own
# Percentage of client's total_hectares_client that can be taken for a SINGLE loan
MIN_LOAN_HECTAR_PORTION = 0.10
MAX_LOAN_HECTAR_PORTION = 1.00

# Maximum attempts to find an eligible client for a new loan in historic generation
MAX_ATTEMPTS_TO_FIND_CLIENT = 1000

# Global tracking for default limits (per culture/season)
# CULTURE_LOANS_IN_PROGRESS = {crop: {'total': 0, 'defaults': 0} for crop in ALL_CROPS}
# DEFAULT_RATE_LIMIT = 0.06 # 6% limit for defaults per culture

# Debt distribution
SHORT_TERM_DEBT_PORTION = 0.40 # ~40% of total debt
MEDIUM_TERM_DEBT_PORTION = 0.30 # ~30% of total debt
LONG_TERM_DEBT_PORTION = 0.30 # ~30% of total debt

# Hectare Rented Configuration
PERCENTAGE_CLIENTS_WITH_RENTED_HECTARE = 0.20 # Max 20% of clients have hectare_rented > 0


In [13]:
# --- Helper Functions ---

def _calculate_loan_amount(hectar, culture):
    """Calculates loan amount based on hectares and crop type."""
    return round(hectar * CROP_VALUES.get(str(culture), 0), 2)

In [14]:
def _determine_rating(score):
    """Determines credit rating based on FICO score."""
    if score >= 800: # Updated score threshold
        return 5  # AAA
    elif score >= 700:
        return 4  # AA
    elif score >= 600:
        return 3  # A
    elif score >= 450: # Updated score threshold
        return 2  # B
    elif score >= 300: # Updated score threshold
        return 1  # C
    else:
        return 0  # D (High Risk)

In [15]:
def _simulate_score_change(current_score, had_previous_default):
    """Simulates score change for the NEXT loan based on previous default status."""
    if had_previous_default:
        # Score drops more significantly after a default
        score_change = random.randint(-120, -10) # Can drop by up to 120 points
    else:
        # Normal fluctuation, slight chance of increase/decrease
        score_change = random.randint(-20, 40) # Can increase slightly more than decrease

    new_score = max(100, min(900, current_score + score_change))
    return new_score

In [16]:
def _simulate_payment_delay_days(score, had_previous_default, culture):
    """
    Simulates the number of days payment is delayed (can be negative for early payment).
    Lower score and previous default increase chance of delay.
    Includes logic to limit overall default rate per culture.
    """
    delay_prob = 0
    if score < 300: # Updated score threshold
        delay_prob = 0.32 # High chance of significant delay
    elif score < 450: # Updated score threshold
        delay_prob = 0.17 # Moderate chance
    elif score < 600:
        delay_prob = 0.08 # Low chance
    elif score < 700:
        delay_prob = 0.03 # Low chance
    else: # score >= 700
        delay_prob = 0.01 # Very low chance

    if had_previous_default:
        delay_prob = min(1.0, delay_prob * 1.5) # Increase delay probability if previous default

    # Check global default limit for the specific culture
    # current_defaults = CULTURE_LOANS_IN_PROGRESS[culture]['defaults']
    # total_loans = CULTURE_LOANS_IN_PROGRESS[culture]['total']

    # If adding a new default would exceed the limit, reduce delay probability for this instance
    # if total_loans > 0 and (current_defaults + 1) / (total_loans + 1) > DEFAULT_RATE_LIMIT:
        # delay_prob *= 0.1 # Significantly reduce delay probability if nearing limit

    if random.random() < delay_prob:
        # Client delays payment
        if score < 300: # Updated score threshold
            return random.randint(20, 120) # More significant delay for lowest scores
        elif score < 450: # Updated score threshold
            return random.randint(10, 60)
        elif score < 600:
            return random.randint(5, 30)
        else: # score >= 600
            return random.randint(1, 15) # Small delay for higher scores
    else:
        # Client pays on time or early
        # Updated score threshold for early/on-time payment
        if score >= 800: # Very low chance of delay, generally early payment
            return random.randint(-10, 3)
        else:
            return random.randint(-5, 5) # Slightly less early for lower scores, but still mostly on time

In [17]:
# --- Main Data Generation Functions ---

def generate_first_transactions(num_unique_clients: int, init_number_id: int = 0):
    """
    Generates the first transaction for a specified number of unique clients.
    Returns a list of dictionaries and a dictionary of client states.
    """
    first_transactions_data = []
    # client_states now tracks hectares_used per season and their due dates
    client_states = {} 

    # Track how many clients have rented hectares to enforce the 20% rule
    clients_with_rented_hectare_count = 0

    for i in range(init_number_id,num_unique_clients + init_number_id):
        client_id = f"CLIENT_{i:04d}"
        initial_score = random.randint(100, 900) # Updated score range
        total_hectares_client = round(random.uniform(MIN_TOTAL_HECTAR_CLIENT, MAX_TOTAL_HECTAR_CLIENT), 2)

        # Determine hectare_rented based on the 20% rule
        hectare_rented = 0.0
        if clients_with_rented_hectare_count < num_unique_clients * PERCENTAGE_CLIENTS_WITH_RENTED_HECTARE:
            if random.random() < PERCENTAGE_CLIENTS_WITH_RENTED_HECTARE * 2: # Give it a higher chance to hit the 20% limit earlier
                hectare_rented = round(random.uniform(1.0, total_hectares_client), 2)
                clients_with_rented_hectare_count += 1

        # Randomize initial loan date over a few years, ensuring it aligns with a crop month
        start_year = 2020 + random.randint(0, 3)
        initial_month = random.choice([1, 6]) # January or June
        loan_date = datetime(start_year, initial_month, random.randint(1, 28))

        income = round(random.uniform(5000, 500000), 2)
        culture = random.choice(JUNE_CROPS if initial_month == 6 else JANUARY_CROPS)

        # Hectar for this loan is a portion of total_hectares_client
        hectar = round(random.uniform(total_hectares_client * MIN_LOAN_HECTAR_PORTION,
                                      total_hectares_client * MAX_LOAN_HECTAR_PORTION), 2)
        hectar = max(1.0, min(hectar, total_hectares_client)) # Ensure hectar is valid

        loan_amount = _calculate_loan_amount(hectar, culture)
        loan_amount_term = random.choice(LOAN_TERMS)
        loan_due_date = loan_date + timedelta(days=loan_amount_term)

        # Simulate payment delay/early payment
        payment_delay_days = _simulate_payment_delay_days(initial_score, False, culture) # No previous default for first loan
        payment_date = loan_due_date + timedelta(days=payment_delay_days)

        # Determine default status based on payment date vs due date
        default_status = 1 if payment_date > loan_due_date else 0

        # Update global default tracking
        # CULTURE_LOANS_IN_PROGRESS[culture]['total'] += 1
        # if default_status == 1:
            # CULTURE_LOANS_IN_PROGRESS[culture]['defaults'] += 1

        rating = _determine_rating(initial_score)
        total_debt = round(random.uniform(loan_amount, min(loan_amount * 2, income * 3)), 2)
        # Explicitly ensure total_debt is not less than loan_amount (already handled by uniform, but reinforces rule)
        total_debt = max(total_debt, loan_amount)

        # Calculate short, medium, and long-term debt
        short_term_debt = round(total_debt * SHORT_TERM_DEBT_PORTION, 2)
        medium_term_debt = round(total_debt * MEDIUM_TERM_DEBT_PORTION, 2)
        long_term_debt = round(total_debt * LONG_TERM_DEBT_PORTION, 2)

        first_transactions_data.append({
            'id_client': client_id,
            'loan_date': loan_date.strftime('%Y-%m-%d'),
            'loan_due_date': loan_due_date.strftime('%Y-%m-%d'),
            'payment_date': payment_date.strftime('%Y-%m-%d'),
            'income': income,
            'culture': culture,
            'hectar': hectar,
            'total_hectares_client': total_hectares_client,
            'hectare_rented': hectare_rented,
            'loan_amount': loan_amount,
            'loan_amount_term': loan_amount_term,
            'rating': rating,
            'score': initial_score,
            'total_debt': total_debt,
            'short_term_debt': short_term_debt, # New column
            'medium_term_debt': medium_term_debt, # New column
            'long_term_debt': long_term_debt, # New column
            'default_status': default_status,
            'transaction_type': 'first'
        })

        # Initialize client state for tracking hectares used per season
        client_states[client_id] = {
            'score': initial_score,
            'last_default_status': bool(default_status),
            'last_loan_date': loan_date,
            'loans_this_year': {loan_date.year: 1},
            'total_hectares_client': total_hectares_client,
            'hectare_rented': hectare_rented,
            'hectares_used_june_season': hectar if culture in JUNE_CROPS else 0,
            'june_loan_due_date': loan_due_date if culture in JUNE_CROPS else None,
            'hectares_used_january_season': hectar if culture in JANUARY_CROPS else 0,
            'january_loan_due_date': loan_due_date if culture in JANUARY_CROPS else None,
        }
    return first_transactions_data, client_states

In [18]:
def generate_historic_transactions(
    first_transactions_data: list,
    client_states: dict,
    num_total_rows: int
):
    """
    Generates additional historic transactions, applying all new rules.
    """
    all_transactions_data = list(first_transactions_data)
    eligible_clients = list(client_states.keys())
    transactions_generated = len(first_transactions_data)
    
    attempts = 0 # To prevent infinite loops if clients run out of options

    while transactions_generated < num_total_rows and attempts < MAX_ATTEMPTS_TO_FIND_CLIENT * (num_total_rows - transactions_generated):
        client_id = random.choice(eligible_clients)
        client_state = client_states[client_id]
        
        # Advance loan_date realistically, ensuring it's after the last loan and in a valid month
        # Start a new transaction 1 month to 1 year after the last loan, plus random days
        base_future_date = client_state['last_loan_date'] + timedelta(days=random.randint(MIN_LOAN_INTERVAL_DAYS, 365))
        
        # Determine the month for the next loan (June or January)
        next_loan_month = random.choice([1, 6])
        
        # Set the loan_date to the determined month in the base_future_date's year
        loan_date = datetime(base_future_date.year, next_loan_month, random.randint(1, 28))

        # Ensure loan_date is always after the last loan for the same client
        if loan_date <= client_state['last_loan_date']:
            # If generated date is not sufficiently after last loan, push it forward
            loan_date = client_state['last_loan_date'] + timedelta(days=MIN_LOAN_INTERVAL_DAYS)
            # Re-adjust month if it changed and needs to be January/June
            if loan_date.month not in [1, 6]:
                loan_date = datetime(loan_date.year, next_loan_month, random.randint(1, 28))
                if loan_date <= client_state['last_loan_date']: # Ensure it's still after if year changed
                    loan_date = datetime(loan_date.year + 1, next_loan_month, random.randint(1, 28))

        # --- Free up hectares from matured loans ---
        if client_state['june_loan_due_date'] and loan_date > client_state['june_loan_due_date']:
            client_state['hectares_used_june_season'] = 0
            client_state['june_loan_due_date'] = None
        if client_state['january_loan_due_date'] and loan_date > client_state['january_loan_due_date']:
            client_state['hectares_used_january_season'] = 0
            client_state['january_loan_due_date'] = None

        # --- Check loan limits for the year ---
        current_year = loan_date.year
        client_state['loans_this_year'].setdefault(current_year, 0) # Ensure year is in dict

        if client_state['loans_this_year'][current_year] >= MAX_LOANS_PER_YEAR:
            attempts += 1
            continue # Try next client/iteration

        # --- Determine available hectares for new loan based on season ---
        culture = random.choice(JUNE_CROPS if loan_date.month == 6 else JANUARY_CROPS)
        is_june_season_loan = culture in JUNE_CROPS
        
        if is_june_season_loan:
            available_hectares_for_this_loan = client_state['total_hectares_client'] - client_state['hectares_used_june_season']
            # If there's an active loan for the same season, ensure it's truly free
            if client_state['hectares_used_june_season'] > 0 and client_state['june_loan_due_date'] and loan_date <= client_state['june_loan_due_date']:
                # Loan for this season is still active, cannot take another one for the same season
                attempts += 1
                continue
        else: # January season loan
            available_hectares_for_this_loan = client_state['total_hectares_client'] - client_state['hectares_used_january_season']
            # If there's an active loan for the same season, ensure it's truly free
            if client_state['hectares_used_january_season'] > 0 and client_state['january_loan_due_date'] and loan_date <= client_state['january_loan_due_date']:
                # Loan for this season is still active, cannot take another one for the same season
                attempts += 1
                continue
        
        if available_hectares_for_this_loan < 1.0: # Need at least 1 hectare to make a loan
            attempts += 1
            continue

        # --- Determine hectar for the new loan ---
        hectar = round(random.uniform(available_hectares_for_this_loan * MIN_LOAN_HECTAR_PORTION,
                                      available_hectares_for_this_loan * MAX_LOAN_HECTAR_PORTION), 2)
        hectar = max(1.0, min(hectar, available_hectares_for_this_loan)) # Ensure valid range

        if hectar < 1.0: # Minimum hectare for a loan
            attempts += 1
            continue

        # --- Generate other transaction details ---
        current_score_for_this_loan = _simulate_score_change(client_state['score'], client_state['last_default_status'])
        client_state['score'] = current_score_for_this_loan # Update client's score for future transactions

        income = round(random.uniform(5000, 500000), 2)
        loan_amount = _calculate_loan_amount(hectar, culture)
        loan_amount_term = random.choice(LOAN_TERMS)
        loan_due_date = loan_date + timedelta(days=loan_amount_term)

        payment_delay_days = _simulate_payment_delay_days(current_score_for_this_loan, client_state['last_default_status'], culture)
        payment_date = loan_due_date + timedelta(days=payment_delay_days)

        default_status = 1 if payment_date > loan_due_date else 0
        rating = _determine_rating(current_score_for_this_loan)
        total_debt = round(random.uniform(loan_amount, min(loan_amount * 2, income * 3)), 2)
        total_debt = max(total_debt, loan_amount) # Ensure total_debt is not less than loan_amount

        # Calculate short, medium, and long-term debt
        short_term_debt = round(total_debt * SHORT_TERM_DEBT_PORTION, 2)
        medium_term_debt = round(total_debt * MEDIUM_TERM_DEBT_PORTION, 2)
        long_term_debt = round(total_debt * LONG_TERM_DEBT_PORTION, 2)

        # Update global default tracking
        # CULTURE_LOANS_IN_PROGRESS[culture]['total'] += 1
        # if default_status == 1:
        #     CULTURE_LOANS_IN_PROGRESS[culture]['defaults'] += 1

        # --- Update client state after successful loan generation ---
        client_state['last_default_status'] = bool(default_status)
        client_state['last_loan_date'] = loan_date
        client_state['loans_this_year'][current_year] += 1

        if is_june_season_loan:
            client_state['hectares_used_june_season'] = hectar
            client_state['june_loan_due_date'] = loan_due_date
        else:
            client_state['hectares_used_january_season'] = hectar
            client_state['january_loan_due_date'] = loan_due_date

        all_transactions_data.append({
            'id_client': client_id,
            'loan_date': loan_date.strftime('%Y-%m-%d'),
            'loan_due_date': loan_due_date.strftime('%Y-%m-%d'),
            'payment_date': payment_date.strftime('%Y-%m-%d'),
            'income': income,
            'culture': culture,
            'hectar': hectar,
            'total_hectares_client': client_state['total_hectares_client'],
            'hectare_rented': client_state['hectare_rented'],
            'loan_amount': loan_amount,
            'loan_amount_term': loan_amount_term,
            'rating': rating,
            'score': current_score_for_this_loan,
            'total_debt': total_debt,
            'short_term_debt': short_term_debt, # New column
            'medium_term_debt': medium_term_debt, # New column
            'long_term_debt': long_term_debt, # New column
            'default_status': default_status,
            'transaction_type': 'historic'
        })
        transactions_generated += 1
        attempts = 0 # Reset attempts after a successful transaction

    if transactions_generated < num_total_rows:
        print(f"Warning: Could only generate {transactions_generated} rows out of {num_total_rows} due to client eligibility constraints and max attempts.")

    return all_transactions_data

In [19]:
# --- Execution ---

def main(num_total_rows: int = 20000, num_unique_clients: int = 1000, init_number_id: int = 0):
    """
    Main function to orchestrate data generation and return DataFrames.
    """
    print(f"Generating {num_unique_clients} first transactions...")
    first_transactions_list, client_states = generate_first_transactions(num_unique_clients, init_number_id)
    df_first_transactions = pd.DataFrame(first_transactions_list)
    
    print(f"Generating {num_total_rows - num_unique_clients} historic transactions...")
    all_transactions_list = generate_historic_transactions(
        first_transactions_list,
        client_states,
        num_total_rows
    )
    df_historic_transactions = pd.DataFrame(all_transactions_list)

    # Convert date columns to datetime objects for proper sorting and analysis
    df_historic_transactions['loan_date'] = pd.to_datetime(df_historic_transactions['loan_date'])
    df_historic_transactions['loan_due_date'] = pd.to_datetime(df_historic_transactions['loan_due_date'])
    df_historic_transactions['payment_date'] = pd.to_datetime(df_historic_transactions['payment_date'])
    
    # Sort historic transactions by client and date to ensure chronological order
    df_historic_transactions = df_historic_transactions.sort_values(by=['id_client', 'loan_date']).reset_index(drop=True)

    print("\nData generation complete!")
    print(f"First transactions DataFrame has '{len(df_first_transactions)}' rows.")
    print(f"Historic transactions DataFrame has '{len(df_historic_transactions)}' rows.")


    return df_first_transactions, df_historic_transactions

In [20]:
# Adjust the total number of rows and unique clients here
df_first, df_historic = main(num_total_rows=30000, num_unique_clients=8000, init_number_id = 0)


Generating 8000 first transactions...
Generating 22000 historic transactions...

Data generation complete!
First transactions DataFrame has '8000' rows.
Historic transactions DataFrame has '30000' rows.


In [21]:
df_historic.to_csv('historic.csv', index=False)

In [22]:
# df_historic.to_excel('historic.xlsx', index=False, sheet_name='Sheet1')